In [1]:
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [2]:
#Read in full dataset
data = pd.read_csv('sentences.csv',
                            sep='\t',
                            encoding='utf8',
                            index_col=0,
                            names=['lang','text'])

In [3]:
#Filter by text length
len_cond = [True if 20<=len(s)<=200 else False for s in data['text']]
data = data[len_cond]

#Filter by text language
lang = ['deu', 'eng', 'fra', 'ita', 'por', 'spa']
data = data[data['lang'].isin(lang)]

#Select 50000 rows for each language
data_trim = pd.DataFrame(columns=['lang','text'])

for l in lang:
    lang_trim = data[data['lang'] ==l].sample(50000,random_state = 100)
    data_trim = data_trim.append(lang_trim)

#Create a random train, valid, test split
data_shuffle = data_trim.sample(frac=1)

train = data_shuffle[0:270000]
#valid = data_shuffle[210000:270000]
test = data_shuffle[270000:300000]

C:\Users\Marc\AppData\Local\Temp\ipykernel_20840\1310559539.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_trim = data_trim.append(lang_trim)
C:\Users\Marc\AppData\Local\Temp\ipykernel_20840\1310559539.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_trim = data_trim.append(lang_trim)
C:\Users\Marc\AppData\Local\Temp\ipykernel_20840\1310559539.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_trim = data_trim.append(lang_trim)
C:\Users\Marc\AppData\Local\Temp\ipykernel_20840\1310559539.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_trim = data_trim.append(lang_trim)
C:\Users\Marc\AppData\Local\Temp

In [4]:
def get_trigrams(corpus,n_feat=200):
    """
    Returns a list of the N most common character trigrams from a list of sentences
    params
    ------------
        corpus: list of strings
        n_feat: integer
    """

    #fit the n-gram model
    vectorizer = CountVectorizer(analyzer='char',
                            ngram_range=(3, 3)
                            ,max_features=n_feat)

    X = vectorizer.fit_transform(corpus)

    #Get model feature names
    feature_names = vectorizer.get_feature_names()

    return feature_names

In [5]:
#obtain trigrams from each language
features = {}
features_set = set()

for l in lang:

    #get corpus filtered by language
    corpus = train[train.lang==l]['text']

    #get 200 most frequent trigrams
    trigrams = get_trigrams(corpus)

    #add to dict and set
    features[l] = trigrams
    features_set.update(trigrams)

#create vocabulary list using feature set
vocab = dict()
for i,f in enumerate(features_set):
    vocab[f]=i

C:\Users\Marc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
#train count vectoriser using vocabulary
vectorizer = CountVectorizer(analyzer='char',
                             ngram_range=(3, 3),
                            vocabulary=vocab)

#create feature matrix for training set
corpus = train['text']
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()

train_feat = pd.DataFrame(data=X.toarray(),columns=feature_names)
train_feat['lang'] = list(train['lang'])

#train_feat.head()

In [7]:
#format data for calculation
chances = {}

for l in lang:
    temp = train_feat[train_feat.lang==l].sum(axis=0)
    temp = temp.drop("lang")
    chances[l] = temp
    chances[l]["total"] = (sum(chances[l]))
#print(chances)

In [8]:
 #Calculate percentage of occurences per trigram
for (key1, keyname) in chances.items():
    for (key2, valuedic) in chances[key1].items():
        chances[key1][key2] = valuedic / chances[key1]["total"]
#print(chances)

Prediction from here!!!!!!!!!!!



In [9]:
 def predictchance(inputSentence):

    lanChances = {}
    for l in lang:
        temp = 0
        for tri in inputSentence:
            try:
                temp += chances[l][tri]
            except:
                pass
        lanChances[l] = temp
    return lanChances

In [10]:
def keywithmaxval(d):
     """ a) create a list of the dict's keys and values;
         b) return the key with the max value"""
     v = list(d.values())
     k = list(d.keys())
     return k[v.index(max(v))]

In [11]:
#input sentence
#languages ['deu', 'eng', 'fra', 'ita', 'por', 'spa']
#accuracy[correct, wrong]
accuracy = [0,0]
for l in lang:
    inputSentence = test[test.lang==l]['text']

    for i in inputSentence:
        inputSentenceTri = get_trigrams([i])
        #print(i)
        #print(inputSentenceTri)
        lanChances = predictchance(inputSentenceTri)
        #print(f"Prediction: {keywithmaxval(lanChances)}, Actual Language: {l}")
        if keywithmaxval(lanChances) == l:
            accuracy[0] +=1
        else:
            accuracy[1] +=1


C:\Users\Marc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
inputSentence = "Your own sentence can be placed here."
inputSentenceTri = get_trigrams([inputSentence])
lanChances = predictchance(inputSentenceTri)

In [13]:
print(lanChances)
print(keywithmaxval(lanChances))

{'deu': 0.04190228894945974, 'eng': 0.07601221714897016, 'fra': 0.061236242641412864, 'ita': 0.043504284588450064, 'por': 0.042220646102308884, 'spa': 0.05019970951343499}
eng


In [14]:
accuracyPercent = (accuracy[0] / (accuracy[1] + accuracy[0]))
print(accuracy)
print(accuracyPercent)

[27414, 2586]
0.9138
